In [1]:
import paddle
from paddle import nn

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [2]:
L = nn.Linear(256, 2048)
L2 = nn.Linear(2048, 256)

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
import numpy as np
import torch


In [4]:
x = np.random.randn(2, 51, 256)
print(x.dtype)
px = paddle.to_tensor(x, dtype='float32')
tx = torch.tensor(x, dtype=torch.float32)
print(px)
print(tx)

float64
Tensor(shape=[2, 51, 256], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[[-0.03137276,  0.75036579, -0.62955737, ..., -0.39516482,  2.41965628,  0.19466873],
         [ 0.55916852,  1.13357353,  0.28754908, ...,  0.28860641,  0.48257691, -1.07664418],
         [-0.27433595, -0.05911482,  0.04942252, ...,  0.46596146,  1.24395037, -1.98374581],
         ...,
         [-0.45322138,  0.51459873,  0.28475651, ..., -0.90797561, -0.80436397, -2.30388594],
         [ 0.20310247,  1.90435207, -1.02483511, ..., -1.59850407, -0.30733466,  0.49769276],
         [-2.63085651, -0.52244109,  0.32019949, ...,  1.10662329, -0.55995786, -0.36770794]],

        [[-1.78831303,  2.24759626,  0.41386250, ..., -0.30020580, -0.16084948,  0.93251175],
         [ 0.03264519, -0.92942363,  1.58523536, ...,  1.23681784, -0.94711000,  0.63553023],
         [-0.19725564, -2.38587499, -0.29334834, ...,  0.83498263, -0.58492625,  0.58732986],
         ...,
         [-0.61646742, -1.02978027

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
data = np.load('enc_0_ff_out.npz', allow_pickle=True)
t_norm_ff = data['norm_ff']
t_ff_out = data['ff_out']
t_ff_l_x = data['ff_l_x']
t_ff_l_a_x = data['ff_l_a_x']
t_ff_l_a_l_x = data['ff_l_a_l_x']
t_ps = data['ps']

In [7]:
L.set_state_dict({'weight': t_ps[0].T, 'bias': t_ps[1]})
L2.set_state_dict({'weight': t_ps[2].T, 'bias': t_ps[3]})

ps = []
for n, p in L.named_parameters():
   ps.append(p)

for n, p in L2.state_dict().items():
    ps.append(p)
    
for p, tp in zip(ps, t_ps):
    print(np.allclose(p.numpy(), tp.T))

True
True
True
True


In [11]:
# data = np.load('enc_0_ff_out.npz', allow_pickle=True)
# t_norm_ff = data['norm_ff']
# t_ff_out = data['ff_out']
# t_ff_l_x = data['ff_l_x']
# t_ff_l_a_x = data['ff_l_a_x']
# t_ff_l_a_l_x = data['ff_l_a_l_x']
# t_ps = data['ps']
TL = torch.nn.Linear(256, 2048)
TL2 = torch.nn.Linear(2048, 256)
TL.load_state_dict({'weight': torch.tensor(t_ps[0]), 'bias': torch.tensor(t_ps[1])})
TL2.load_state_dict({'weight': torch.tensor(t_ps[2]), 'bias': torch.tensor(t_ps[3])})

# for n, p in TL.named_parameters():
#    print(n, p)
# for n, p in TL2.named_parameters():
#    print(n, p)

ps = []
for n, p in TL.state_dict().items():
    ps.append(p.data.numpy())
    
for n, p in TL2.state_dict().items():
    ps.append(p.data.numpy())
    
for p, tp in zip(ps, t_ps):
    print(np.allclose(p, tp))

True
True
True
True


In [12]:
y = L(px)
print(y.numpy())

ty = TL(tx)
print(ty.data.numpy())
print(np.allclose(px.numpy(), tx.detach().numpy()))
print(np.allclose(y.numpy(), ty.detach().numpy()))

[[-0.25528666 -0.9090747   0.12996    ...  0.02552819  0.37376517
  -0.558986  ]
 [-0.45657372  0.23811203  0.33472425 ...  1.0797666  -0.7263612
   0.31549692]]
[[-0.25528657 -0.9090746   0.12996009 ...  0.02552832  0.37376505
  -0.5589858 ]
 [-0.45657367  0.23811209  0.33472428 ...  1.0797666  -0.7263612
   0.31549698]]
True
False


In [14]:
x = np.random.randn(2, 256)
px = paddle.to_tensor(x, dtype='float32')
tx = torch.tensor(x, dtype=torch.float32)
y = L(px)
print(y.numpy())
ty = TL(tx)
print(ty.data.numpy())
print(np.allclose(px.numpy(), tx.detach().numpy()))
print(np.allclose(y.numpy(), ty.detach().numpy()))
print(np.allclose(y.numpy(), ty.detach().numpy(), atol=1e-5))

[[ 0.07453135  0.0698561   0.6273111  ...  0.5845924  -0.65527105
   0.5881643 ]
 [ 0.3902049  -0.17455879 -1.1802813  ... -0.36912322  0.55681896
  -0.11917676]]
[[ 0.07453132  0.06985616  0.62731117 ...  0.5845925  -0.65527105
   0.5881642 ]
 [ 0.39020485 -0.17455864 -1.1802814  ... -0.3691232   0.556819
  -0.11917675]]
True
False
True
